In [1]:
from glob import glob
import numpy as np
from structs import read_map
import subprocess
import pandas as pd
import os

In [45]:
try:
    #os.mkdir("obs/")
    #os.mkdir("obs/rooms/")
    os.mkdir("obs/random/")
except:
    pass

In [50]:
!mkdir obs/dao/

In [47]:
map_sources = [
    #"../../hog2/maps/rooms/32room_*.map",
    #"../../hog2/maps/random/random512*.map",
    #"../../hog2/maps/mazes/maze512*.map"
]

def mapn_to_obsn(filepath):
    return filepath.replace("../../hog2/maps/", "obs/").replace(".map", "*.obs")
    

In [2]:
maps = [
    "../../hog2/maps/rooms/32room_004.map",
    "../../hog2/maps/random/random512-20-1.map",
    "../../hog2/maps/dao/den520d.map"
]
scen = {
    "../../hog2/maps/rooms/32room_004.map": "../../hog2/scenarios/rooms/32room_004.map.scen" ,
    "../../hog2/maps/random/random512-20-1.map": "../../hog2/scenarios/random/random512-20-1.map.scen",
    "../../hog2/maps/dao/den520d.map": "../../hog2/scenarios/dao/den520d.map.scen"
}
  
#for source in map_sources:
#    maps += glob(source)

In [51]:
for m in maps:
    for n in range(16):
        print(m, n)
        outfname = m.replace("../../hog2/maps/", "obs/").replace(".map", "_16384_" + str(n) + ".obs")
        if not os.path.exists(outfname) or True:
            command = [
                "./genobs.py", m, "16384", "1", outfname
            ]
            subprocess.run(command)
            for i in ["0", "4", "16", "64", "256", "1024", "4096"]:
                command = "head -" + i + " " + outfname + " > " + outfname.replace("_16384_", "_" + i + "_")
                subprocess.run(command, shell = True)

../../hog2/maps/rooms/32room_004.map 0
../../hog2/maps/rooms/32room_004.map 1
../../hog2/maps/rooms/32room_004.map 2
../../hog2/maps/rooms/32room_004.map 3
../../hog2/maps/rooms/32room_004.map 4
../../hog2/maps/rooms/32room_004.map 5
../../hog2/maps/rooms/32room_004.map 6
../../hog2/maps/rooms/32room_004.map 7
../../hog2/maps/rooms/32room_004.map 8
../../hog2/maps/rooms/32room_004.map 9
../../hog2/maps/rooms/32room_004.map 10
../../hog2/maps/rooms/32room_004.map 11
../../hog2/maps/rooms/32room_004.map 12
../../hog2/maps/rooms/32room_004.map 13
../../hog2/maps/rooms/32room_004.map 14
../../hog2/maps/rooms/32room_004.map 15
../../hog2/maps/random/random512-20-1.map 0
../../hog2/maps/random/random512-20-1.map 1
../../hog2/maps/random/random512-20-1.map 2
../../hog2/maps/random/random512-20-1.map 3
../../hog2/maps/random/random512-20-1.map 4
../../hog2/maps/random/random512-20-1.map 5
../../hog2/maps/random/random512-20-1.map 6
../../hog2/maps/random/random512-20-1.map 7
../../hog2/maps/ra

In [78]:
with open("new_partial_commands.run", "wt") as comf:
    allendt = "5000"
    for m in maps: 
        grid = read_map(m)
        s = scen[m]
        scenarios = pd.read_csv(s, sep = "\t",index_col = 0, comment = "v", names = ["Bucket","map","map width","map height","start x-coordinate","start y-coordinate","goal x-coordinate","goal y-coordinate","optimal length"])
        scenarios = scenarios.sort_values("optimal length", ascending = False)
        for i in range(16):
            print(i)
            scenario = scenarios.iloc[i]
            for obs in glob(mapn_to_obsn(m)):
                if ("1024" not in obs):
                    continue
                for search in ["partialpdap"]:
                    command = ["../build_release/pdap",
                               "--search " + search,
                               "--obstacles " + obs,
                               "--map " + m,
                               "--startx " + str(scenario["start x-coordinate"]),
                               "--starty " + str(scenario["start y-coordinate"]),
                               "--goalx " + str(scenario["goal x-coordinate"]),
                               "--goaly " + str(scenario["goal y-coordinate"]),
                               "--startt 0",
                               "--startendt 10",
                               "--maxquery",
                               "--allendt", allendt,
                               " > " + "results/" + "_".join([search, obs.split("/")[-1], m.split("/")[-1],
                                                 str(scenario["start x-coordinate"]),  str(scenario["start y-coordinate"]),
                                                              str(scenario["goal x-coordinate"]), str(scenario["goal y-coordinate"]), 
                                                 allendt, ".res"])
                              ]
                    print(" ".join(command), file = comf)
    
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [79]:
!wc new_commands.run

   9216  211968 2635008 new_commands.run


In [75]:
obs

'obs/dao/den520d_1024_2.obs'